## Read file

In [1]:
import numpy as np
import pandas as pd

class ProductionData:
    def __init__(self, demand_forecast, setup_cost, production_cost, holding_cost, num_periods):
        self.demand_forecast = np.array(demand_forecast)
        self.setup_cost = np.array(setup_cost)
        self.production_cost = np.array(production_cost)
        self.holding_cost = np.array(holding_cost)
        self.num_periods = num_periods
    
    def __repr__(self):
        return (f"ProductionData(\n"
                f"  Demand Forecast: {self.demand_forecast}\n"
                f"  Setup Cost: {self.setup_cost}\n"
                f"  Production Cost: {self.production_cost}\n"
                f"  Holding Cost: {self.holding_cost}\n"
                f"  Number of Periods: {self.num_periods}\n")

def read_production_data(file_path, sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Insert dummy period 0
    demand_forecast = np.insert(df["Demand Forecast"].to_numpy(), 0, 0)
    setup_cost = np.insert(df["Setup Cost"].to_numpy(), 0, 0)
    production_cost = np.insert(df["Production cost"].to_numpy(), 0, 0)
    holding_cost = np.insert(df["Holding cost"].to_numpy(), 0, 0)
    
    num_periods = df.shape[0]
    
    return ProductionData(demand_forecast, setup_cost, production_cost, holding_cost, num_periods)

# Define file path and sheet names
file_path = "ULSP-instancesR.xlsx"
sheet_list = ["6-periods (1)", "6-periods (2)", "12-periods (1)", "12-periods (2)", 
              "24-periods (1)", "24-periods (2)", "52-periods (1)", "52-periods (2)", 
              "104-periods (1)", "104-periods (2)"]



## 1 MILP AND LP solutions (Gurobi)

### 1.1 MILP

In [ ]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np


# DataFrame to store all results
results_df = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S"])

# Process each sheet
for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")

    # Read the Excel file from the specific sheet
    data=read_production_data(file_path, sheet_name)
    T=data.num_periods
    
    # Create Gurobi model
    model = gp.Model("Uncapacitated Lot-Sizing Problem")

    # Decision variables
    y = model.addVars(T+1, vtype=GRB.BINARY, name="y")  
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")  
    S = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="S")  

    # Objective function
    model.setObjective(
        quicksum(data.setup_cost[t] * y[t] for t in range(1, T+1)) +
        quicksum(data.production_cost[t] * x[t] for t in range(1, T+1)) +
        quicksum(data.holding_cost[t] * S[t] for t in range(1, T+1)),
        GRB.MINIMIZE
    )

    # Constraints
    model.addConstr(S[0] == 0, name="no_inventory0")
    model.addConstr(S[T] == 0, name="no_inventoryT")

    for t in range(1, T+1):
        model.addConstr(x[t] + S[t-1] == data.demand_forecast[t] + S[t], name=f"demand_satisfied_{t}")
        model.addConstr(x[t] <= quicksum(data.demand_forecast[m] for m in range(t, T+1)) * y[t], name=f"setup_constraint_{t}")

    # Solve model
    model.optimize()

    if model.status == GRB.OPTIMAL:

        print(f"{'Period':<10}{'y':<10}{'x':<10}{'S':<15}")
        for t in range(1, T+1):
            y_val = y[t].X
            x_val = x[t].X
            S_val = S[t].X

            # Print results for the current period
            print(f"{t:<10}{y_val:<10}{x_val:<15}{S_val:<15}")

            # Append to DataFrame
            results_df = pd.concat([results_df, pd.DataFrame({
                "Sheet": [sheet_name],
                "Period": [t],
                "y": [y_val],
                "x": [x_val],
                "S": [S_val]
            })], ignore_index=True)
        
        
# Print all results at the end
print("\nAll results:")
print(results_df)

# Optionally save to a CSV file
results_df.to_csv("non_optimized_results.csv", index=False)
print("\nResults saved to 'non_optimized_results.csv'.")



Processing sheet: 6-periods (1)
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 21 columns and 32 nonzeros
Model fingerprint: 0x0695a2d9
Variable types: 14 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 7 rows and 10 columns
Presolve time: 0.01s
Presolved: 7 rows, 11 columns, 17 nonzeros
Variable types: 7 continuous, 4 integer (4 binary)
Found heuristic solution: objective 54875.000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

C:\Users\ricca\AppData\Local\Temp\ipykernel_18400\2379512754.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({



12        1.0       863.0          0.0            

Processing sheet: 12-periods (2)
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 39 columns and 62 nonzeros
Model fingerprint: 0x08bd9955
Variable types: 26 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [2e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 1e+03]
Found heuristic solution: objective 45218.000000
Presolve removed 6 rows and 8 columns
Presolve time: 0.00s
Presolved: 20 rows, 31 columns, 50 nonzeros
Variable types: 20 continuous, 11 integer (11 binary)

Root relaxation: objective 4.151611e+04, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds   

### 2. LP-Relaxation vs MILP solutions

In [ ]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np
import time

# Function to solve ULSP for LP relaxation and MILP
def solve_ulsp(data, relax_y=False):
    T = data.num_periods
    model = gp.Model("ULSP")
    
    # Decision variables
    y = model.addVars(T+1, vtype=GRB.CONTINUOUS if relax_y else GRB.BINARY, lb=0, ub=1, name="y")
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")  
    S = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="S")  
    
    # Objective function
    model.setObjective(
        quicksum(data.setup_cost[t] * y[t] for t in range(1, T+1)) +
        quicksum(data.production_cost[t] * x[t] for t in range(1, T+1)) +
        quicksum(data.holding_cost[t] * S[t] for t in range(1, T+1)),
        GRB.MINIMIZE
    )
    
    # Constraints
    model.addConstr(S[0] == 0, name="no_inventory0")
    model.addConstr(S[T] == 0, name="no_inventoryT")
    
    for t in range(1, T+1):
        model.addConstr(x[t] + S[t-1] == data.demand_forecast[t] + S[t], name=f"demand_satisfied_{t}")
        model.addConstr(x[t] <= quicksum(data.demand_forecast[m] for m in range(t, T+1)) * y[t], name=f"setup_constraint_{t}")
    
    # Solve model
    start_time = time.time()
    model.optimize()
    solve_time = time.time() - start_time
    
    # Store results
    obj_value = model.objVal if model.status == GRB.OPTIMAL else None
    return obj_value, solve_time, model

# Data storage
results = []

# Process each sheet
for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")
    
    # Read data
    data = read_production_data(file_path, sheet_name)
    
    # Solve LP relaxation
    lp_obj, lp_time, lp_model = solve_ulsp(data, relax_y=True)
    
    # Solve MILP
    milp_obj, milp_time, milp_model = solve_ulsp(data, relax_y=False)
    
    # Store results
    results.append([sheet_name, lp_obj, lp_time, milp_obj, milp_time])
    
# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=["Sheet", "LP Objective", "LP Time", "MILP Objective", "MILP Time"])

# Print and save results
print("\nFinal Results:")
print(results_df)
results_df.to_csv("lp_vs_milp_nobacklog_results.csv", index=False)
print("Results saved to 'lp_vs_milp_results.csv'.")


## 3.1 Pattern Observation in MILP

The first observation is that the Setups are minimized by clustering them together(at-least for the selected instances), as the model aims to reduce the frequency of costly setups(setup costs are really high), leading to grouped production that satisfies future demand while keeping inventory costs low. This patterns—setup minimization is fundamental to achieving optimal solutions in this problem, ensuring that the model always finds an efficient balance between setup costs, production costs, and inventory holding costs.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the all_results CSV file into a DataFrame
results_df = pd.read_csv("non_optimized_results.csv")

# Define function to plot data
def plot_data(data, sheet_name):
    fig, ax1 = plt.subplots(figsize=(12, 4))
    
    ax1.bar(data['Period'], data['x'], color='g', width=0.4, label='Production Quantity (x)', alpha=0.6)
    ax1.plot(data['Period'], data['S'], marker='s', linestyle='-', color='r', label='Inventory Level (S)', alpha=0.7)
    ax1.set_xlabel('Period')
    ax1.set_ylabel('x (Production Quantity) / S (Inventory Level)')
    ax1.tick_params(axis='y')

    ax2 = ax1.twinx()
    ax2.plot(data['Period'], data['y'], marker='o', linestyle='-', color='b', label='Setup Decision (y)', alpha=0.7)
    ax2.set_ylabel('y (Binary)', color='b')
    ax2.tick_params(axis='y', labelcolor='b')

    plt.title(f'Production and Inventory Levels with Setup Decision for {sheet_name}')
    ax1.grid(True)
    
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# spliting the CSV into instances
sheet_ranges = [
    ("6-periods (1)", 0, 6),
    ("6-periods (2)", 6, 12),
    ("12-periods (3)", 12, 24),
    ("12-periods (4)", 24, 36)
]

for sheet_name, start_row, end_row in sheet_ranges:
    sheet_data = results_df.iloc[start_row:end_row] 
    plot_data(sheet_data, sheet_name)

## 3.2 Wagner within

In [37]:
import numpy as np

def wagner_whitin_nobacklog(demand, setup_costs, holding_costs, unit_costs, num_periods):
    T = num_periods  # Number of periods
    C = np.full(T+1, float('inf'))  # Cost array, initialized to infinity
    C[0] = 0  # Base case: No cost at period 0
    order_periods = [-1] * (T+1)  # To track order placements
    
    # For each period t
    for t in range(1, T+1):
        # Try all possible last order periods j to find which j to produce to meet the demand
        for j in range(1, t+1): 
            # Calculate costs when producing in period j for periods j through t
            
            # Setup cost for production in period j
            cost = setup_costs[j]
            
            # Production cost for all demand from j to t
            production_cost = unit_costs[j] * sum(demand[k] for k in range(j, t+1)) #produce whatever required till t
            cost += production_cost
            
            # Holding costs for demands of periods k (j < k ≤ t)
            holding_cost = sum(
                holding_costs[k] * sum(demand[m] for m in range(k+1, t+1))
                for k in range(j, t)
            )
            cost += holding_cost
            
            # Add cost of optimal solution up to j-1
            total_cost = C[j-1] + cost #sum it to the cost of previous period
            
            # Update if this solution is better (minimum cost for j period)
            if total_cost < C[t]:
                C[t] = total_cost
                order_periods[t] = j
    
    # Backtrack to find optimal order schedule
    t = T
    orders = []
    while t > 0:
        j = order_periods[t]
        order_quantity = sum(demand[k] for k in range(j, t+1))
        orders.append((j, order_quantity))
        t = j-1
    
    return list(reversed(orders)), C[T]

results1=[]
    
for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")

    # Read the Excel file from the specific sheet
    data=read_production_data(file_path, sheet_name)
    #print(data.demand_forecast[0])
    #print(data.holding_cost[0])
    start_time=time.time()
    optimal_orders, min_cost = wagner_whitin_nobacklog(data.demand_forecast, data.setup_cost, data.holding_cost, data.production_cost, data.num_periods)
    total_time=time.time()-start_time

    results1.append([sheet_name, min_cost, total_time])
    
    print("Optimal order schedule:", optimal_orders)
    print("Minimum total cost:", min_cost)

# Convert results to DataFrame
results_df = pd.DataFrame(results1, columns=["Sheet", "Minimun cost", "Processing time"])

results_df.to_csv("Wagner_within_formulation.csv", index=False)









Processing sheet: 6-periods (1)
Optimal order schedule: [(1, 185), (2, 1312), (6, 542)]
Minimum total cost: 54875.0

Processing sheet: 6-periods (2)
Optimal order schedule: [(1, 573), (2, 3808)]
Minimum total cost: 105079.0

Processing sheet: 12-periods (1)
Optimal order schedule: [(1, 892), (3, 325), (5, 477), (6, 817), (7, 627), (9, 487), (11, 506), (12, 863)]
Minimum total cost: 54291.0

Processing sheet: 12-periods (2)
Optimal order schedule: [(1, 623), (2, 906), (4, 958), (5, 319), (7, 689), (8, 414), (9, 372), (10, 346), (12, 938)]
Minimum total cost: 44793.0

Processing sheet: 24-periods (1)
Optimal order schedule: [(1, 660), (2, 837), (3, 164), (5, 857), (6, 190), (9, 100), (10, 454), (11, 924), (12, 496), (13, 703), (14, 469), (15, 706), (16, 907), (17, 321), (19, 192), (22, 883), (23, 783), (24, 727)]
Minimum total cost: 78316.0

Processing sheet: 24-periods (2)
Optimal order schedule: [(1, 1067), (3, 1366), (5, 853), (7, 695), (11, 842), (13, 1245), (16, 958), (18, 585), (1

## 4. Verify inequalities

In [ ]:

# Read the Excel file from the specific sheet
data=read_production_data(file_path,"6-periods (1)" )
T=data.num_periods
    
# Create Gurobi model
model = gp.Model("Uncapacitated Lot-Sizing Problem")

# Decision variables as before ofc 
y = model.addVars(T+1, vtype=GRB.BINARY, name="y")  
x = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")  
S = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="S")  

# Objective function
model.setObjective(
    quicksum(data.setup_cost[t] * y[t] for t in range(1, T+1)) +
    quicksum(data.production_cost[t] * x[t] for t in range(1, T+1)) +
    quicksum(data.holding_cost[t] * S[t] for t in range(1, T+1)),
    GRB.MINIMIZE
)

# Constraints
model.addConstr(S[0] == 0, name="no_inventory0")
model.addConstr(S[T] == 0, name="no_inventoryT")

for t in range(1, T+1):
    model.addConstr(x[t] + S[t-1] == data.demand_forecast[t] + S[t])
    model.addConstr(x[t] <= quicksum(data.demand_forecast[m] for m in range(t, T+1)) * y[t])

#in this case we don't care about previous objective function, we care about checking validity of new ineq for all the variables in the polyhedron
# Define l and S for the inequality check
l = T  # Last period (you can change this dynamically)
S_subset = list(range(1, l+1))  # Example subset

# Compute D_tl values, l is fixed, tehre is a D_tl for each t, looks like it's a matrix
D = [[0] * (T+1) for x in range(T+1)] 
for l_val in range(1, T+1):  # Loop over l
    for t in range(1, l_val+1):  # Loop over t from 1 to l
        D[t][l_val] = sum(data.demand_forecast[i] for i in range(t, l_val+1))

#all constrints set

model.setParam(GRB.Param.PoolSearchMode, 2)  # Explore entire feasible region
model.setParam(GRB.Param.PoolSolutions, 10000000)  # Allow up to 1M solutions (adjust if needed)

model.optimize()

num_solutions = model.SolCount  # Get the number of feasible solutions found
print(f"Total feasible integer solutions found: {num_solutions}")


model.optimize()

print(model.solcount)


integer_solutions = []
for i in range(model.SolCount):
    model.setParam(GRB.Param.SolutionNumber, i)
    y_values = model.getAttr("Xn", y)

    # Check if all y-values are integer
    if all(abs(y_values[t] - round(y_values[t])) < 1e-6 for t in range(1, T+1)):
        integer_solutions.append(y_values)
# Iterate through the feasible solutions Gurobi found

N=len(integer_solutions)


# Verify the inequality for all integer solutions
for i in range(N):  
    y_values = integer_solutions[i]  # Directly use stored values

    # Extract x and S values
    x_values = model.getAttr("X", x)  
    S_values = model.getAttr("X", S)  

    # Compute left-hand and right-hand side for the inequality
    for l_val in range(1, T+1):
        lhs = sum(x_values[t] for t in range(1, l_val+1))
        rhs = sum(D[t][l_val] * y_values[t] for t in range(1, l_val+1)) + S_values[l_val]
        
        if lhs > rhs:
            print(f" Inequality violated at solution {i}: {lhs} > {rhs}")
            break  # Stop checking if one violation is found          
            
print("valid inequalities")
    
    

## 5. Separation technique

In [19]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np

# DataFrame to store all results
results_df = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S"])


sheet_list = ["104-periods (1)"]
              
#, "6-periods (2)", "12-periods (1)", "12-periods (2)", 
 #             "24-periods (1)", "24-periods (2)", "52-periods (1)", "52-periods (2)", 
  #            "104-periods (1)", "104-periods (2)"]


# Process each sheet
for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")
    
    # Read the Excel file from the specific sheet
    data = read_production_data(file_path, sheet_name)
    T = data.num_periods
    
    # Create Gurobi model
    model = gp.Model("Uncapacitated Lot-Sizing Problem")
    
    # Decision variables
    y = model.addVars(T+1, vtype=GRB.BINARY, name="y")  
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")  
    S = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="S")  
    
    # Objective function
    model.setObjective(
        quicksum(data.setup_cost[t] * y[t] for t in range(1, T+1)) +
        quicksum(data.production_cost[t] * x[t] for t in range(1, T+1)) +
        quicksum(data.holding_cost[t] * S[t] for t in range(1, T+1)),
        GRB.MINIMIZE
    )

    # Constraints
    model.addConstr(S[0] == 0, name="no_inventory0")
    model.addConstr(S[T] == 0, name="no_inventoryT")

    # Relax binary constraints on y to be continuous in [0,1]
    for t in range(1, T+1):
        y[t].vtype = GRB.CONTINUOUS  # Convert binary to continuous
        y[t].lb = 0  # Lower bound
        y[t].ub = 1  # Upper bound


    for t in range(1, T+1):
        model.addConstr(x[t] + S[t-1] == data.demand_forecast[t] + S[t], name=f"demand_satisfied_{t}")
        model.addConstr(x[t] <= quicksum(data.demand_forecast[m] for m in range(t, T+1)) * y[t], name=f"setup_constraint_{t}")

    # Solve initial LP relaxation
    model.Params.InfUnbdInfo = 1  # Ensure feasibility insights
    model.optimize()
    # Retrieve LP-relaxed solution
    x_vals = {t: x[t].X for t in range(1, T+1)}
    y_vals = {t: y[t].X for t in range(1, T+1)}
    S_vals = {t: S[t].X for t in range(1, T+1)}
    
    # Print LP relaxation results
    print("\nLP Relaxation Solution:")
    for t in range(1, T+1):
        print(f"Period {t}: y={y_vals[t]:.4f}, x={x_vals[t]:.4f}, S={S_vals[t]:.4f}")

    while True:
        # Get LP relaxation solution, yeah
        x_vals = {t: x[t].X for t in range(1, T+1)}
        y_vals = {t: y[t].X for t in range(1, T+1)}
        S_vals = {t: S[t].X for t in range(1, T+1)}

        # Separation procedure: Find the most violated (l, S) inequality
        most_violated = None
        max_violation = 0
        
        for l in range(1, T+1): #for each subset, we are finding the most violated
            
            S_l = [j for j in range(1, l+1) if x_vals[j] - sum(data.demand_forecast[j:l+1]) * y_vals[j] > 0]
            
            if not S_l:
                continue
            
            v_l = sum(x_vals[t] - sum(data.demand_forecast[t:l+1]) * y_vals[t] for t in S_l) - S_vals[l]
            
            if v_l > max_violation:
                max_violation = v_l
                most_violated = (l, S_l)

        # If no violated inequalities, stop
        if max_violation <= 0:
            break
        
        # Add the most violated (l, S) inequality
        l_star, S_star = most_violated
        model.addConstr(quicksum(x[t] for t in S_star) <= quicksum(sum(data.demand_forecast[t:l_star+1]) * y[t] for t in S_star) + S[l_star], name=f"(l,S)_ineq_{l_star}")
        
        # Re-optimize with new constraints
        model.optimize()
    
    # Store final results
    if model.status == GRB.OPTIMAL:
        for t in range(1, T+1):
            results_df = pd.concat([results_df, pd.DataFrame({
                "Sheet": [sheet_name],
                "Period": [t],
                "y": [y[t].X],
                "x": [x[t].X],
                "S": [S[t].X]
            })], ignore_index=True)

# Print results
print("\nAll results:")
print(results_df)

# Save results
results_df.to_csv("separation_results.csv", index=False)
print("\nResults saved to 'all_results.csv'.")    

C:\Users\ricca\AppData\Local\Temp\ipykernel_30832\1495588366.py:106: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({


## 7. Facility location reformulation (FLPR).

Now, define the variable wqt to be the amount produced in period q (q <= t) and used to meet the 
demand of the item in period t; that is the fraction of the demand of the item in period t that is 
produced in period q. Reformulate the single-item uncapacitated lot-sizing problem using only wqt and 
yt variables. This reformulation is called the facility location reformulation (FLPR)


I produce in q to satisfy in period t

demand of period t can be satisfied by different periods q


### LOOKS LIKE THE PRODUCTION QUANTITIES DO NOT CHANGE

In [28]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np
from pydantic import constr
import matplotlib.pyplot as plt
import time


# DataFrame to store all results
results_df = pd.DataFrame(columns=["Sheet", "Period", "w", "y",])

sheet_list = ["6-periods (1)", "6-periods (2)", "12-periods (1)", "12-periods (2)", 
              "24-periods (1)", "24-periods (2)", "52-periods (1)", "52-periods (2)", 
              "104-periods (1)", "104-periods (2)"]

def solve_FLPR(data, relax_y=False):
    T = data.num_periods
    model = gp.Model("Lot Sizing - Facility Location")
    
    # Decision variables
    # y[j]: 1 if we produce in period j (like opening a facility)
    # w[j,i]: fraction of demand i satisfied by production in period j
    y = model.addVars(T+1, vtype=GRB.CONTINUOUS if relax_y else GRB.BINARY, name="y")#so we define them two as before
    w = model.addVars(T+1, T+1, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="w")
    
    # Objective function
    model.setObjective(
    # Setup costs
    quicksum(data.setup_cost[j] * y[j] for j in range(1, T+1)) +
    # Production costs
    quicksum(
        quicksum(
            data.production_cost[i] * data.demand_forecast[j] * w[i,j] 
            for j in range(i, T+1)
        ) for i in range(1, T+1)
    ) +
    # Holding costs 
    quicksum(
        quicksum(
            sum(data.holding_cost[t] for t in range(i, j)) * data.demand_forecast[j] * w[i,j]
            for j in range(i+1, T+1)  
        ) for i in range(1, T+1)
    ),
    GRB.MINIMIZE
    )

    # Each demand must be fully satisfied (like each customer must be served)
    for j in range(1, T+1):
        model.addConstr(
            quicksum(w[i,j] for i in range(1, j+1)) == 1, #yeah
            name=f"demand_{j}"
        )
    

    for j in range(1, T+1):
        for i in range(1, j+1):
            model.addConstr( w[i,j]<= y[i]) 

    
    # Solve
    start_time = time.time()
    model.optimize()
    solve_time = time.time() - start_time
    
    return model.objVal if model.status == GRB.OPTIMAL else None, solve_time, model    # Store results in DataFrame
    
#let's store results
results = []

for sheet_name in sheet_list:
    print(f"processing sheet: {sheet_name}")

    data=read_production_data(file_path, sheet_name)

    # Solve LP relaxation
    lp_obj, lp_time, lp_model = solve_FLPR(data, relax_y=True)
    
    # Solve MILP
    milp_obj, milp_time, milp_model = solve_FLPR(data, relax_y=False)
    
    gap=abs(((milp_obj-lp_obj)/milp_obj)*100)
    
    # Store results
    results.append([sheet_name, lp_obj, lp_time, milp_obj, milp_time, gap])

 
    
# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=["Sheet", "LP Objective", "LP Time", "MILP Objective", "MILP Time", "Gap%"])
    

# Print and save results
results_df.to_csv("FLPR_results.csv", index=False)
print("\nResults saved to 'FLPR_results.csv'.")


processing sheet: 6-periods (1)
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 27 rows, 56 columns and 63 nonzeros
Model fingerprint: 0x927e7159
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+02, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 27 rows and 56 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.4875000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  5.487500000e+04
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz